In [1]:
import pandas as pd
import numpy as np

In [80]:
def analyze(filename):
    df = pd.read_csv(filename, header=-1)
    df.columns = ['pred', 'label'] # Distribution of templates
    
    for id in df['label'].unique():
        subset = df[df['label'] == id]
        correct = (subset['pred'] == subset['label']).sum()
        accuracy = correct / len(subset)

        false_positive_subset = df[df['pred'] == id]
        false_positives = (false_positive_subset['pred'] != false_positive_subset['label']).sum()
        false_positive_rate = false_positives / max(len(false_positive_subset), 1)

        print("=" * 50)
        print("Template #{}: count={}, correct={}, false_positives={}, accuracy={}, false_positive_rate={}"
              .format(id, len(subset), correct, false_positives, accuracy, false_positive_rate))
        print("=" * 50)

In [81]:
analyze("analysis/input_dim=300,mem_dim=150,epochs=15,current_epoch=11,train_acc=0.70375.csv")

Template #101: count=62, correct=32, false_positives=19, accuracy=0.5161290322580645, false_positive_rate=0.37254901960784315
Template #301: count=249, correct=180, false_positives=146, accuracy=0.7228915662650602, false_positive_rate=0.44785276073619634
Template #2: count=597, correct=528, false_positives=201, accuracy=0.8844221105527639, false_positive_rate=0.2757201646090535
Template #105: count=138, correct=111, false_positives=74, accuracy=0.8043478260869565, false_positive_rate=0.4
Template #16: count=409, correct=371, false_positives=67, accuracy=0.9070904645476773, false_positive_rate=0.15296803652968036
Template #305: count=462, correct=268, false_positives=98, accuracy=0.5800865800865801, false_positive_rate=0.2677595628415301
Template #5: count=158, correct=52, false_positives=50, accuracy=0.3291139240506329, false_positive_rate=0.49019607843137253
Template #3: count=208, correct=129, false_positives=66, accuracy=0.6201923076923077, false_positive_rate=0.3384615384615385
Tem